In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model
%aimport allel.bcolz
allel.__version__

'0.15.0.dev9'

In [2]:
# setup an array of genotype calls
shape = n_variants, n_samples, ploidy = 50000, 1000, 2
data = np.zeros(shape, dtype='i1')
# simulate some mutations
n_alleles = n_variants * n_samples * ploidy
idx = np.random.randint(0, (n_alleles//2)-1, size=n_alleles//20)
data[:, :, 1].reshape((-1))[idx] = 1
data[:, :, 0].reshape((-1))[idx[:n_alleles//200]] = 1
g = allel.model.GenotypeArray(data, copy=False)

In [3]:
g

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/1,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/1,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/1,...,0/0,0/0,0/0,0/0,0/0


In [4]:
g.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [5]:
h = g.to_haplotypes()
h

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,1,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,0,0,0,0


In [6]:
h.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [7]:
import bcolz

In [8]:
gc = allel.bcolz.GenotypeCArray(g, cparams=bcolz.cparams(cname='zlib', clevel=1))
gc

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/1,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/1,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/1,...,0/0,0/0,0/0,0/0,0/0


In [9]:
gc.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [10]:
hc = gc.to_haplotypes()
hc

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,1,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,0,0,0,0


In [11]:
hc.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [12]:
ac = g.count_alleles()
ac

,0,1
0,1915,85
1,1915,85
2,1897,103
3,1895,105
4,1898,102


In [13]:
gac = gc.count_alleles()
gac

,0,1
0,1915,85
1,1915,85
2,1897,103
3,1895,105
4,1898,102


In [14]:
gc.is_called()

CArrayWrapper((50000, 1000), bool)
  nbytes: 47.68 MB; cbytes: 748.94 KB; ratio: 65.20
  cparams := cparams(clevel=1, shuffle=True, cname='zlib')
[[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]]

In [15]:
gc.to_n_alt()

CArrayWrapper((50000, 1000), int8)
  nbytes: 47.68 MB; cbytes: 7.68 MB; ratio: 6.21
  cparams := cparams(clevel=1, shuffle=True, cname='zlib')
[[0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]]